# Introduction

Considering that I'm an urban safety planner in Japan, and I'm analyzing which areas of Japan need extra earthquake reinforcement.  Specifically the areas that are both high in population density and prone to earthquakes.

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/Kuh9gPj.png" width="450"><br/>
</center>

In [1]:
import pandas as pd
import geopandas as gpd

import folium
from folium import Choropleth
from folium.plugins import HeatMap

#### Define a function `embed_map()` for displaying interactive maps. It accepts two arguments: the variable containing the map, and the name of the HTML file where the map will be saved.

This function ensures that the maps are visible in all web browsers

In [2]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

## Do earthquakes coincide with plate boundaries?

In [6]:
plate_boundaries = gpd.read_file("Plate_Boundaries.shp")
plate_boundaries['coordinates'] = plate_boundaries.apply(lambda x: [(b,a) for (a,b) in list(x.geometry.coords)], axis='columns')
plate_boundaries.drop('geometry', axis=1, inplace=True)

plate_boundaries.head()

,HAZ_PLATES,HAZ_PLAT_1,HAZ_PLAT_2,Shape_Leng,coordinates
0,TRENCH,SERAM TROUGH (ACTIVE),6722,5.843467,"[(-5.444200361999947, 133.6808931800001), (-5...."
1,TRENCH,WETAR THRUST,6722,1.829013,"[(-7.760600482999962, 125.47879802900002), (-7..."
2,TRENCH,TRENCH WEST OF LUZON (MANILA TRENCH) NORTHERN ...,6621,6.743604,"[(19.817899819000047, 120.09999798800004), (19..."
3,TRENCH,BONIN TRENCH,9821,8.329381,"[(26.175899215000072, 143.20620700100005), (26..."
4,TRENCH,NEW GUINEA TRENCH,8001,11.998145,"[(0.41880004000006466, 132.8273013480001), (0...."


In [7]:
plate_boundaries['coordinates'].iloc[0]

[(-5.444200361999947, 133.6808931800001),
 (-5.193599878999976, 133.597396902),
 (-5.0409002209999585, 133.50700362100008),
 (-4.87900020099994, 133.3043975060001),
 (-4.75809997999994, 133.18879702300012),
 (-4.587100180999926, 133.14649969800007),
 (-4.388599878999969, 133.04930094500003),
 (-4.282499496999947, 132.99319452800012),
 (-4.200299737999956, 132.88220177000005),
 (-4.119299939999962, 132.70480386200006),
 (-4.018800039999974, 132.56289680100008),
 (-3.780200160999925, 132.35350030200004),
 (-3.586100381999927, 132.1681975460001),
 (-3.433000401999948, 131.95300342000007),
 (-3.1976996579999764, 131.63040132800006),
 (-2.9921001809999552, 131.3282931000001),
 (-2.8492003619999764, 131.01820358100008),
 (-2.723500301999934, 130.75729330100012),
 (-2.6297998389999293, 130.4815067390001),
 (-2.597800240999959, 130.22860692000006),
 (-2.5657996379999304, 129.9391017900001),
 (-2.5223003419999372, 129.60499597700004),
 (-2.4697002619999466, 129.33039831000008),
 (-2.43889961799

loading the historical earthquake data into a DataFrame `earthquakes`.

In [8]:
# Load the data
earthquakes = pd.read_csv("earthquakes1970-2014.csv", parse_dates=["DateTime"])
earthquakes.head()

,DateTime,Latitude,Longitude,Depth,Magnitude,MagType,NbStations,Gap,Distance,RMS,Source,EventID
0,1970-01-04 17:00:40.200,24.139,102.503,31.0,7.5,Ms,90.0,NaN,NaN,0.0,NEI,1.970010e+09
1,1970-01-06 05:35:51.800,-9.628,151.458,8.0,6.2,Ms,85.0,NaN,NaN,0.0,NEI,1.970011e+09
2,1970-01-08 17:12:39.100,-34.741,178.568,179.0,6.1,Mb,59.0,NaN,NaN,0.0,NEI,1.970011e+09
3,1970-01-10 12:07:08.600,6.825,126.737,73.0,6.1,Mb,91.0,NaN,NaN,0.0,NEI,1.970011e+09
4,1970-01-16 08:05:39.000,60.280,-152.660,85.0,6.0,ML,0.0,NaN,NaN,NaN,AK,NaN


### Visualizing the plate boundaries on a map. Using all of the earthquake data to add a heatmap to the same map, to determine whether earthquakes coincide with plate boundaries.  

In [9]:
# Create a base map with plate boundaries
m_1 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_1)

# Adding earthquakes heatmap to the plate boundaries map
HeatMap(data=earthquakes[['Latitude','Longitude']],radius=15).add_to(m_1)

# Show the map
embed_map(m_1, 'q_1.html')

![Q1](q1.jpg)


[Click here for the above map](q_1.html)

### From the above map we can see that earthquakes coincide with plate boundaries.

## Is there a relationship between earthquake depth and proximity to a plate boundary in Japan?

I'm interested to see if there are any intereresting global patterns, and I'd also like to understand how depth varies in Japan.



In [10]:
earthquakes['Depth'].describe()

count    5304.000000
mean       74.268037
std       129.092258
min         0.000000
25%        14.500000
50%        33.000000
75%        52.300000
max       688.000000
Name: Depth, dtype: float64

In [12]:
# Create a base map with plate boundaries
m_2 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_2)
    
# map to visualize earthquake depth
def color_producer(val):
    if val < 50:
        return 'green'
    elif val < 250:
        return 'orange'
    else:
        return 'red'

for i in range(len(earthquakes)):
    folium.Circle(location=[earthquakes.iloc[i]['Latitude'],earthquakes.iloc[i]['Longitude']],radius=500,color=color_producer(earthquakes.iloc[i]['Depth'])).add_to(m_2)

# View the map
embed_map(m_2, 'q_2.html')

![Q1](q2.jpg)

[Click here for the above map](q_2.html)

### In the northern half of Japan, it does appear that earthquakes closer to plate boundaries tend to be shallower (and earthquakes farther from plate boundaries are deeper). This pattern is repeated in other locations, such as the western coast of South America. But, it does not hold everywhere (for instance, in China, Mongolia, and Russia).

## Which prefectures have high population density?

In [14]:
# GeoDataFrame with prefecture boundaries
prefectures = gpd.read_file("japan-prefecture-boundaries/japan-prefecture-boundaries.shp")
prefectures.set_index('prefecture', inplace=True)
prefectures.head()

,geometry
prefecture,
Aichi,"MULTIPOLYGON (((137.09523 34.65330, 137.09546 ..."
Akita,"MULTIPOLYGON (((139.55725 39.20330, 139.55765 ..."
Aomori,"MULTIPOLYGON (((141.39860 40.92472, 141.39806 ..."
Chiba,"MULTIPOLYGON (((139.82488 34.98967, 139.82434 ..."
Ehime,"MULTIPOLYGON (((132.55859 32.91224, 132.55904 ..."


The next code cell creates a DataFrame `stats` containing the population, area (in square kilometers), and population density (per square kilometer) for each Japanese prefecture.  Run the code cell without changes.

In [15]:
# DataFrame containing population of each prefecture
population = pd.read_csv("japan-prefecture-population.csv")
population.set_index('prefecture', inplace=True)

# area (in square kilometers) of each prefecture
area_sqkm = pd.Series(prefectures.geometry.to_crs(epsg=32654).area / 10**6, name='area_sqkm')
stats = population.join(area_sqkm)

# density (per square kilometer) of each prefecture
stats['density'] = stats["population"] / stats["area_sqkm"]
stats.head()

,population,area_sqkm,density
prefecture,,,
Tokyo,12868000,1800.614782,7146.448049
Kanagawa,8943000,2383.038975,3752.771186
Osaka,8801000,1923.151529,4576.342460
Aichi,7418000,5164.400005,1436.372085
Saitama,7130000,3794.036890,1879.264806


### Choropleth map to visualize population density.

In [16]:
# Create a base map
m_3 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

# choropleth map to visualize population density
Choropleth(geo_data=prefectures['geometry'].__geo_interface__, 
           data=stats['density'], 
           key_on="feature.id", 
           fill_color='YlGnBu', 
           legend_name='Population Density(per sqkm)'
          ).add_to(m_3)

# View the map
embed_map(m_3, 'q_3.html')

![Q1](q3.jpg)

[Click here for the above map](q_3.html)

### The prefectures of Tokyo, Yokohama, and Osaka have the highest population density. All of these prefectures are located in central Japan, and Tokyo and Yokohama are adjacent.

## Which high-density prefecture is prone to high-magnitude earthquakes?

In [17]:
earthquakes['Magnitude'].describe()

count    5304.000000
mean        6.369114
std         0.419054
min         6.000000
25%         6.100000
50%         6.200000
75%         6.500000
max         9.100000
Name: Magnitude, dtype: float64

In [24]:
# Create a base map
m_4 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

# create a function to assign a color to a map
def color_producer(magnitude):
    if magnitude > 6.5:
        return '#bf1029'
    else:
        return '#4a9c54'
# choropleth map to visualize earthquake magnitude
Choropleth(
    geo_data=prefectures['geometry'].__geo_interface__,
    data=stats['density'],
    key_on="feature.id",
    fill_color='BuPu',
    legend_name='Population density (per square kilometer)').add_to(m_4)
# add markers to the map
for i in range(0,len(earthquakes)):
    folium.Circle(
        location=[earthquakes.iloc[i]['Latitude'], earthquakes.iloc[i]['Longitude']],
        popup=("{} ({})").format(
            earthquakes.iloc[i]['Magnitude'],
            earthquakes.iloc[i]['DateTime'].year),
        radius=earthquakes.iloc[i]['Magnitude']**5.5,
        color=color_producer(earthquakes.iloc[i]['Magnitude'])).add_to(m_4)

# View the map
embed_map(m_4, 'q_4.html')

![Q1](q4.jpg)

[Click here for the above map](q_4.html)

### **Tokyo** is by far the most densely populated prefecture and has also experienced a number of earthquakes.  **Osaka** is relatively less densely populated, but experienced an earthquake that was relatively stronger than those near Tokyo.  And, **Yokohama** (in addition to its high density and the historical proximity of strong earthquakes) might lead us to worry about the added potential tsunami risk.


## I would recommend **Yokohama** and **Osaka** for extra earthquake reinforcement. As they both have experienced strong earthquakes, they are relatively close to each other. I wouldn't recommend **Tokyo** for extra earthquake reinforcement eventhough it being densely populated because it has experienced weaker earthquakes compared to **Yokohama** and **Osaka**.
